In [101]:
import numpy as np
s = 1
d = 0
n_ex = 1
out_rows = 7
out_cols = 7
out_ch = 5

fr = 3
fc = 3

in_rows = out_rows + fr - 1 
in_cols = out_cols + fc - 1
in_ch = 3

# simple test 1
# W = (np.array(range(fr*fc*in_ch*out_ch)) + 1).reshape(fr, fc, in_ch, out_ch)
# X = (np.array(range(n_ex*in_rows*in_cols*in_ch)) + 1).reshape(n_ex, in_rows, in_cols, in_ch)
# dZ = (np.array(range(n_ex*out_rows*out_cols*out_ch)) + 1).reshape(n_ex, out_rows, out_cols, out_ch)

# simple test 2
# W = np.ones(fr*fc*in_ch*out_ch).reshape(fr, fc, in_ch, out_ch)
# X = 2*np.ones(n_ex*in_rows*in_cols*in_ch).reshape(n_ex, in_rows, in_cols, in_ch)
# dZ = np.ones(n_ex*out_rows*out_cols*out_ch).reshape(n_ex, out_rows, out_cols, out_ch)

# random int test 
W = np.random.randint(10, size=(fr*fc*in_ch*out_ch)).reshape(fr, fc, in_ch, out_ch)
X = np.random.randint(10, size=(n_ex*in_rows*in_cols*in_ch)).reshape(n_ex, in_rows, in_cols, in_ch)
dZ = np.random.randint(10, size=(n_ex*out_rows*out_cols*out_ch)).reshape(n_ex, out_rows, out_cols, out_ch)

# real test 
# W = np.random.rand(fr*fc*in_ch*out_ch).reshape(fr, fc, in_ch, out_ch)
# X = np.random.rand(n_ex*in_rows*in_cols*in_ch).reshape(n_ex, in_rows, in_cols, in_ch)
# dZ = np.random.rand(n_ex*out_rows*out_cols*out_ch).reshape(n_ex, out_rows, out_cols, out_ch)

In [102]:
# filter 0
print(W[:, :, 0, 0]) 
print(W[:, :, 1, 0])

[[4 5 2]
 [9 6 7]
 [9 7 4]]
[[4 8 0]
 [3 0 7]
 [8 9 6]]


In [103]:
# filter 1
print(W[:, :, 0, 1]) 
print(W[:, :, 1, 1])

[[8 6 2]
 [9 3 6]
 [1 2 8]]
[[5 6 1]
 [6 1 6]
 [8 9 6]]


In [104]:
# ifmap
print(X[0, :, :, 0]) 
print(X[0, :, :, 1])

[[8 4 3 5 1 7 9 4 6]
 [9 0 1 1 2 5 1 3 2]
 [0 0 4 4 3 8 1 6 6]
 [8 8 2 8 3 6 5 5 0]
 [2 3 9 3 3 3 6 0 8]
 [0 3 3 3 4 2 5 2 1]
 [1 2 9 2 2 2 5 4 8]
 [5 1 2 1 2 1 8 1 7]
 [4 5 6 0 5 2 6 7 1]]
[[1 1 1 1 3 6 0 2 9]
 [0 8 9 5 2 2 7 8 9]
 [6 1 7 8 5 3 1 9 4]
 [5 3 9 7 3 1 7 6 9]
 [5 1 0 1 3 5 0 9 3]
 [4 7 3 4 9 6 9 8 2]
 [0 8 9 6 7 5 7 8 8]
 [1 2 7 3 5 4 8 4 3]
 [9 8 4 3 6 7 9 1 9]]


In [105]:
# upstream gradients
print(dZ[0, :, :, 0])
print(dZ[0, :, :, 1])

[[1 7 3 3 0 0 1]
 [6 9 7 5 8 6 0]
 [5 8 7 1 4 2 0]
 [2 0 0 7 0 1 1]
 [5 2 1 1 9 4 0]
 [6 1 4 4 6 1 4]
 [2 1 7 9 6 3 5]]
[[4 3 2 7 3 8 4]
 [7 9 6 1 2 8 7]
 [9 2 2 2 0 9 9]
 [6 4 5 7 2 6 8]
 [6 5 5 0 1 2 5]
 [4 1 4 9 8 9 2]
 [4 2 0 2 8 1 5]]


In [106]:
# ground truth placeholder
dW = np.zeros_like(W)
dX = np.zeros_like(X)

for m in range(n_ex):
    for i in range(out_rows):
        for j in range(out_cols):
            for c in range(out_ch):
                # compute window boundaries w. stride and dilation
                i0, i1 = i * s, (i * s) + fr * (d + 1) - d
                j0, j1 = j * s, (j * s) + fc * (d + 1) - d

                wc = W[:, :, :, c]
                kernel = dZ[m, i, j, c]
                window = X[m, i0 : i1 : (d + 1), j0 : j1 : (d + 1), :]

                # dB[:, :, :, c] += kernel
                dW[:, :, :, c] += window * kernel
                dX[m, i0 : i1 : (d + 1), j0 : j1 : (d + 1), :] += wc * kernel

In [107]:
# reproducing with deep loop nest
N = n_ex
C = in_ch
K = out_ch
P = out_cols
Q = out_rows
R = fc
S = fr
(N, C, K, P, Q, R, S)

(1, 3, 5, 7, 7, 3, 3)

In [108]:
# test placeholder
test_dW = np.zeros_like(W)
test_dX = np.zeros_like(X)

# gradient wrt filters
for n in range(N):
    for c in range(C):
        for k in range(K):
            for p in range(P):
                for q in range(Q):
                    for r in range(R):
                        for s in range(S):
                            test_dW[s, r, c, k] += X[n, q+s, p+r, c] * dZ[n, q, p, k]

# emmm numerical issue with FP...
dW - test_dW

array([[[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]]])

In [109]:
# gradient wrt to activations
# error gradient map need to be padded
# npad is a tuple of (n_before, n_after) for each dimension
npad = ((0, 0), (fr-1, fr-1), (fc-1, fc-1), (0, 0))
padded_dZ = np.pad(dZ, pad_width=npad, mode='constant', constant_values=0)
# print(padded_dZ[0, :, :, 0])
# print(padded_dZ[0, :, :, 1])

# generate spatially rotated (180 degree) weight
spat_W = np.zeros_like(W)
for i in range(in_ch):
    for j in range(out_ch):
        for s in range(fr):
            for r in range(fc):
                spat_W[fr-1-s, fc-1-r, i, j] = W[s, r, i, j]
print(W[:, :, 0, 0]) 
# print(W[:, :, 1, 0])
print(spat_W[:, :, 0, 0]) 
# print(spat_W[:, :, 1, 0])

[[4 5 2]
 [9 6 7]
 [9 7 4]]
[[4 7 9]
 [7 6 9]
 [2 5 4]]


In [111]:
test_dX = np.zeros_like(X)

for n in range(N):
    for c in range(C):
        for k in range(K):
            for pr in range(P+R-1):
                for qs in range(Q+S-1):
                    for r in range(R):
                        for s in range(S):
                            # test_dX[n, qs, pr, c] += padded_dZ[n, qs+s, pr+r, k] * spat_W[s, r, c, k]
                            test_dX[n, qs, pr, c] += padded_dZ[n, qs+s, pr+r, k] * W[S-1-s, R-1-r, c, k]

# emmm numerical issue with FP...
dX - test_dX

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [

In [80]:
# gradient wrt filter 0
print(dW[:, :, 0, 0]) 
print(dW[:, :, 1, 0])

[[144 147 141]
 [ 79 142 159]
 [105 199 181]]
[[120 102  95]
 [112  77 101]
 [109 116 148]]


In [68]:
# gradient wrt filter 0
print(dW[:, :, 0, 1]) 
print(dW[:, :, 1, 1])

[[120  98 137]
 [ 83 106 127]
 [112 149 134]]
[[ 95  58 114]
 [ 76  54  95]
 [ 80 100 114]]


In [69]:
# gradient wrt ifmap
print(dX[0, :, :, 0]) 
print(dX[0, :, :, 1])

[[ 44  61  27  21]
 [ 78 136  86  62]
 [148 194 141 104]
 [ 96  82  73  49]]
[[ 85  73  56  31]
 [161 164 200 129]
 [136 186 222 128]
 [ 53  79  77  36]]
